In [ ]:
import numpy as np
import pandas as pd

import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

from tqdm import tqdm

In [ ]:
SINAN_PATH      = './../data/raw_data/SINAN_prep_05.csv'
MUNICIPIOS_PATH = './../data/municipios.csv'
OCUPACOES_PATH  = './../data/cbo_ocupacoes.csv'
MAPBOX_TOKEN    = 'pk.eyJ1IjoibHVjYXNuc2VxIiwiYSI6ImNrb241dHZ0cTBpd2MycW5yMGp2enFtMmkifQ.N6NJGlWhG-iYrIJMQ1MVVw'

px.set_mapbox_access_token(MAPBOX_TOKEN)

In [ ]:
SINAN_COLS = [
    'VIOLENCE', 'AGRESSOR', 'MUN_NOME', 'MUN_UF', 'MUN_LATITUDE', 
    'MUN_LONGITUDE', 'MUN_ID', 'MUN_POP_2013', 'MUN_POP_2014', 
    'MUN_POP_2015', 'MUN_POP_2016', 'MUN_POP_2017', 'DT_YEAR',
    'DT_MONTH', 'DT_DAY', 'AGE', 'ESCOLARIDADE', 'CBO_OCUPACAO',
    'CBO_FAMILIA', 'CBO_GRUPO','CS_RACA',
    'CS_SEXO', 'DT_NASC', 'DT_NOTIFIC', 'SIT_CONJUG', 'VIOL_MOTIV',
    'IDENT_GEN', 'ORIENT_SEX','ID_UNIDADE'
    ]

sinan_df      = pd.read_csv(SINAN_PATH, usecols=SINAN_COLS)
# sinan_df      = pd.read_csv(SINAN_PATH)
municipios_df = pd.read_csv(MUNICIPIOS_PATH)

In [ ]:
municipios_df.head(3)

In [ ]:
fig = ff.create_hexbin_mapbox(
    data_frame=municipios_df, lat="latitude", lon="longitude",
    nx_hexagon=45, opacity=0.5, labels={"color": "Municípios"},
    min_count=1,
)
fig.show()

In [ ]:
sinan_df.head(3)

In [ ]:
def filter_by_list(dataframe, column, values):
    
    new_dataframe = dataframe[dataframe[column].isin(values)]
    
    return new_dataframe

def sinan_filtros(dataframe, UF = None):
    
    # UF
    if UF:
        new_dataframe = filter_by_list(dataframe, 'MUN_UF', [UF])
    else:
        new_dataframe = dataframe
    
    # VIOLENCE
    violence_list = ['Física', 'Psicológica', 'Sexual']
    
    new_dataframe = filter_by_list(new_dataframe, 'VIOLENCE', violence_list)
    
    # AGRESSOR
    agressor_list = ['Pai', 'Mãe', 'Padrasto', 'Cônjuge', 
                     'Ex-cônjuge', 'Namorado/a', 'Ex-namorado/a', 
                     'Filho/a','Irmão/ã', 'Amigo/a', 'Madrasta']
    
    new_dataframe = filter_by_list(new_dataframe, 'AGRESSOR', agressor_list)
    
    # SEXO
    
    sexo = "F"
    
    new_dataframe = new_dataframe[new_dataframe['CS_SEXO'] == 'F']
    
    
    
    return new_dataframe

In [ ]:
%%time

sinan_df_selected = sinan_filtros(sinan_df)
sinan_df_selected.shape

In [ ]:
fig = ff.create_hexbin_mapbox(
    data_frame=sinan_df, lat="MUN_LATITUDE", lon="MUN_LONGITUDE",
    nx_hexagon=45, opacity=0.5, labels={"color": "Denúncias"},
    min_count=1,
)
fig.show()

In [ ]:
fig = make_subplots(rows=4, cols=2, subplot_titles=("Violência", "Agressor", "Raça", "Sexo", "Motivo da violência", "Situação conjugal", "", "Idade"))

fig.add_trace(
    go.Histogram(x=sinan_df_selected['VIOLENCE']),
    row=1, col=1
)

fig.add_trace(
    go.Histogram(x=sinan_df_selected['AGRESSOR']),
    row=1, col=2
)

fig.add_trace(
    go.Histogram(x=sinan_df_selected['CS_RACA']),
    row=2, col=1
)

fig.add_trace(
    go.Histogram(x=sinan_df_selected['CS_SEXO']),
    row=2, col=2
)

fig.add_trace(
    go.Histogram(x=sinan_df_selected['VIOL_MOTIV']),
    row=3, col=1
)

fig.add_trace(
    go.Histogram(x=sinan_df_selected['SIT_CONJUG']),
    row=3, col=2
)

fig.add_trace(
    go.Histogram(x=sinan_df_selected['AGE']),
    row=4, col=2
)


fig.update_layout(height=1500, width=800, title_text="Distribuições no Brasil")
fig.update_xaxes(categoryorder="total descending")
fig.show()

In [ ]:
fig = px.histogram(sinan_df_selected, color="AGRESSOR", x="AGE", title=f'Distribuição denúncias por idade/agressor')
fig.show()

In [ ]:
df = sinan_df_selected.drop(sinan_df_selected[sinan_df_selected.SIT_CONJUG.isna()].index)

fig = px.histogram(df, color="SIT_CONJUG", x="AGE", title=f'Distribuição denúncias por idade/sit. conjugal')
fig.show()

In [ ]:
df = sinan_df_selected.drop(sinan_df_selected[sinan_df_selected.SIT_CONJUG.isna()].index)

fig = px.histogram(df, color="SIT_CONJUG", x="ESCOLARIDADE", title=f'Distribuição de nível de escolaridade',
                  category_orders = {'ESCOLARIDADE': ['EF I incompleto','EF I completo','EF II incompleto',
                                                      'EF II completo','EM incompleto','EM completo',
                                                      'ES incompleto','ES completo',]})
fig.show()

In [ ]:
fig = px.histogram(sinan_df_selected, color="CS_SEXO", x="ESCOLARIDADE", title=f'Distribuição de nível de escolaridade',
                  category_orders = {'ESCOLARIDADE': ['EF I incompleto','EF I completo','EF II incompleto',
                                                      'EF II completo','EM incompleto','EM completo',
                                                      'ES incompleto','ES completo',]})
fig.show()

In [ ]:
fig = px.histogram(sinan_df_selected, 
                   y="CBO_FAMILIA", 
                   color="AGRESSOR", 
                   title=f'Distribuição de família de ocupação',
                   height=8000
                    ).update_yaxes(categoryorder="total ascending")
                  
fig.show()

In [ ]:
fig = px.histogram(sinan_df_selected, 
                   y="CBO_GRUPO", 
                   color="AGRESSOR", 
                   title=f'Distribuição de grupos de ocupação').update_yaxes(categoryorder="total ascending")
                  
fig.show()

In [ ]:
def get_encoder(values):
    
    encoder = LabelEncoder()
    encoded = encoder.fit_transform(values.reshape(-1, 1))
    
    return encoded, encoder

In [ ]:
columns         = ['ESCOLARIDADE', 'DT_YEAR', 'DT_MONTH', 'AGE', 
                   'CS_RACA', 'VIOLENCE', 'AGRESSOR', 'CBO_GRUPO', 
                   'CBO_FAMILIA', 'CBO_OCUPACAO', 'SIT_CONJUG', 'VIOL_MOTIV',
                    'IDENT_GEN', 'ORIENT_SEX']

feature_columns = columns[4:]

df = sinan_df_selected[columns]
df = df.dropna()

encoders = {}
encodeds = {}
for column in feature_columns:
    
    encodeds[column], encoders[column] = get_encoder(df[column].values)
    df[f'{column}_ID'] = encodeds[column]


escolaridade_map = {
    'EM completo': 6, 
    'EF II completo': 4, 
    'EF II incompleto': 3,
    'EM incompleto': 5, 
    'ES completo': 8, 
    'ES incompleto': 7, 
    'EF I incompleto': 1,
    'EF I completo': 2
}

def g(val):
    
    try: return escolaridade_map[val]
    except: return None

    
df['ESCOLARIDADE_ID'] = df['ESCOLARIDADE'].apply(g)
df = df.dropna()

In [ ]:
df.head(2).T

In [ ]:
columns = ['ESCOLARIDADE_ID', 'DT_YEAR', 'DT_MONTH', 'AGE', 'CS_RACA_ID', 
           'VIOLENCE_ID', 'AGRESSOR_ID', 'CBO_GRUPO_ID', 'CBO_FAMILIA_ID', 'CBO_OCUPACAO_ID',
           'SIT_CONJUG_ID', 'VIOL_MOTIV_ID', 'IDENT_GEN_ID', 'ORIENT_SEX_ID']


fig = px.imshow(df[columns].corr())
fig.show()

In [ ]:
map_month = {
    1: 'Janeiro',
    2: 'Fevereiro',
    3: 'Março',
    4: 'Abril',
    5: 'Maio',
    6: 'Junho',
    7: 'Julho',
    8: 'Agosto',
    9: 'Setembro',
    10: 'Outubro',
    11: 'Novembro',
    12: 'Dezembro',
}

df = pd.DataFrame()

for year in [2013., 2014., 2015., 2016., 2017.]:
    
    temp = sinan_df_selected[sinan_df_selected['DT_YEAR'] == year]

    temp = temp.groupby('DT_MONTH').count().reset_index()

    temp['MONTH'] = temp['DT_MONTH'].apply(lambda x : map_month[x])
    temp['COUNT'] = temp['AGE']
    
    temp = temp[['MONTH', 'COUNT', 'DT_MONTH']]
    temp['YEAR'] = [year]*len(temp)
    
    df = pd.concat([df, temp])

fig = px.scatter(df, 
                 x="MONTH", y="YEAR", color="COUNT", size="COUNT", 
                 title="Casos por ano/mês no Brasil", 
                 color_continuous_scale='plotly3')

fig.show()

In [ ]:
municipios_df.head()

In [ ]:
sinan_df['AGRESSOR'].unique()

In [ ]:
for year in tqdm([2013, 2014, 2015, 2016, 2017]):
    
    denun_count_values = []
    unit_count_values  = []
    denun_sex_count_values  = []
    denun_maior_count_values  = []
    denun_relacao_count_values  = []

    for mun_id in municipios_df.id.values:

        mun_data = sinan_df_selected[sinan_df_selected['MUN_ID'] == mun_id]
        mun_data = mun_data[mun_data['DT_YEAR'] == year]

        count_unidades      = mun_data.ID_UNIDADE.nunique()
        count_denuncias     = len(mun_data)
        count_denun_sexismo = len(mun_data[mun_data['VIOL_MOTIV'] == 'Sexismo'])
        count_denun_maioridade = len(mun_data[mun_data['AGE'] >= 18])
        count_denun_relacao = len(filter_by_list(mun_data, 'AGRESSOR', ['Namorado/a', 'Cônjuge', 'Ex-cônjuge', 'Ex-namorado/a']))
        
        unit_count_values.append(count_unidades)
        denun_count_values.append(count_denuncias)
        denun_sex_count_values.append(count_denun_sexismo)
        denun_maior_count_values.append(count_denun_maioridade)
        denun_relacao_count_values.append(count_denun_relacao)
    
    municipios_df[f'unit_count_{year}'] = unit_count_values
    municipios_df[f'unit_relat_{year}'] = municipios_df[f'unit_count_{year}']/municipios_df[f'pop_{year}']
    municipios_df[f'denun_count_{year}'] = denun_count_values
    municipios_df[f'denun_relat_{year}'] = municipios_df[f'denun_count_{year}']/municipios_df[f'pop_{year}']
    municipios_df[f'denun_sexismo_count_{year}'] = denun_sex_count_values
    municipios_df[f'denun_sexismo_relat_{year}'] = municipios_df[f'denun_sexismo_count_{year}']/municipios_df[f'pop_{year}']
    municipios_df[f'denun_maioridade_count_{year}'] = denun_maior_count_values
    municipios_df[f'denun_maioridade_relat_{year}'] = municipios_df[f'denun_maioridade_count_{year}']/municipios_df[f'pop_{year}']
    municipios_df[f'denun_relacao_count_{year}'] = denun_relacao_count_values
    municipios_df[f'denun_relacao_relat_{year}'] = municipios_df[f'denun_relacao_count_{year}']/municipios_df[f'pop_{year}']

municipios_df.head()

In [ ]:
municipios_df[municipios_df['nome'] == 'São Paulo']

In [ ]:
municipios_df.sample(3)

In [ ]:
municipios_df.columns

In [ ]:
try:
    values_df = municipios_df.drop(columns=['id', 'nome', 'uf_nome', 'uf_id', 'sinan_id', 'latitude', 'longitude', 'cluster', 'cluster_id'])
except:
    values_df = municipios_df.drop(columns=['id', 'nome', 'uf_nome', 'uf_id', 'sinan_id', 'latitude', 'longitude'])

encoders = {}
encodeds = {}
feature_columns = ['uf', 'regiao']

for column in feature_columns:
    
    encodeds[column], encoders[column] = get_encoder(values_df[column].values)
    values_df[f'{column}_ID'] = encodeds[column]

values_df.head()

In [ ]:
scaler = MinMaxScaler()

values = values_df.drop(columns=feature_columns).values
scaled_values = scaler.fit_transform(values)

# Run the Kmeans algorithm and get the index of data points clusters
sse = []
n_clusters_list = list(range(1, 10))

for n_clusters in n_clusters_list:
    km = KMeans(n_clusters=n_clusters)
    km.fit(scaled_values)
    sse.append(km.inertia_)

cluster_eval_df = pd.DataFrame({'n_clusters': n_clusters_list, 'inertia': sse})

fig = px.line(
    cluster_eval_df,
    x='n_clusters', y='inertia', 
    title='Cluster evaluation')

fig.show()  

In [ ]:
clusterizer = KMeans(n_clusters = 3)
scaler = MinMaxScaler()

values = values_df.drop(columns=feature_columns).values
scaled_values = scaler.fit_transform(values)

clusterizer = clusterizer.fit(scaled_values)

municipios_df['cluster'] = clusterizer.predict(scaled_values)
municipios_df['cluster_id'] = municipios_df['cluster'].apply(lambda x : f'cluster_{x}')

In [ ]:
fig = px.imshow(municipios_df.corr(), height=700)
fig.show()

In [ ]:
df    = pd.DataFrame()
years = [2013, 2014, 2015, 2016, 2017]
num_years = len(years)

for index in tqdm(municipios_df.index):
    
    mun_data = municipios_df.loc[index]
    
    columns = ['unit_count', 'unit_relat', 'denun_count', 'denun_relat', 'pop']
    
    temp_df = pd.DataFrame()
    temp_df['year'] = years
    temp_df['id']   = [mun_data.id] * num_years
    temp_df['cluster_id']   = [mun_data.cluster_id] * num_years
    temp_df['cluster']   = [mun_data.cluster] * num_years
    temp_df['nome'] = [mun_data.nome] * num_years
    temp_df['uf']   = [mun_data.uf] * num_years
    temp_df['regiao']   = [mun_data.regiao] * num_years
    
    for col in columns:
        
        values = []
        
        for year in years:

            col_name = f'{col}_{year}'
            
            value = mun_data[col_name]
            values.append(value)
            
        temp_df[col] = values
        
    df = pd.concat((df, temp_df))    

In [ ]:
df.head()

In [ ]:
fig = px.scatter_matrix(df,
    dimensions=["unit_count", "unit_relat", "denun_count", "denun_relat", 'pop', 'year', 'cluster'],
    color="cluster_id", height=1000)
fig.show()

In [ ]:
fig = px.scatter(
    df[df['pop'] > 30000], 
    x="unit_relat", y="denun_relat", color="cluster_id", 
    title='Unidades x Denuncias | Cluster',
    hover_data=['year', 'nome', 'uf'], range_x=[0, 0.0015], range_y=[0, 0.020])

fig.show()  

In [ ]:
fig = px.scatter(
    df, 
    x="unit_relat", y="denun_relat", color="regiao", 
    title='Unidades x Denuncias',
    hover_data=['year', 'nome', 'uf', 'pop'], range_x=[0, 0.0015], range_y=[0, 0.020])

fig.show()  

In [ ]:
capitals = ['Rio Branco','Maceió','Macapá','Manaus','Salvador',
            'Fortaleza','Brasília','Vitória','Goiânia','São Luís',
            'Cuiabá','Campo Grande','Belo Horizonte','Belém','João Pessoa',
            'Curitiba','Recife','Teresina','Rio de Janeiro','Natal',
            'Porto Alegre','Porto Velho','Boa Vista','Florianópolis',
            'São Paulo','Aracaju','Palmas']

capitals_df = filter_by_list(df, 'nome', capitals)

fig = px.scatter(
    capitals_df, 
    x="unit_relat", y="denun_relat", color="cluster_id", size="pop",
    title='Unidades x Denuncias',
    hover_data=['year', 'uf', 'nome'])

fig.show()  

In [ ]:
px.imshow(capitals_df.corr())

In [ ]:
capitals = ['Ananindeua','Camaçari','Canoas','Juazeiro',
            'Itabuna','Betim','Caruaru','Maceió','Vitória','São Leopoldo', 'Brasília']

capitals_df = filter_by_list(df, 'nome', capitals)

fig = px.scatter(
    capitals_df, 
    x="unit_relat", y="denun_relat", color="nome", size='pop',
    title='Unidades x Denuncias',
    hover_data=['year', 'uf'])

fig.show()  

In [ ]:
capitals = ['AC']

capitals_df = filter_by_list(df, 'uf', capitals)

fig = px.scatter(
    capitals_df, 
    x="unit_relat", y="denun_relat", color="cluster_id", size="pop",
    title='Unidades x Denuncias',
    hover_data=['year', 'uf'])

fig.show()  

In [ ]:
capitals = ['AC']

capitals_df = filter_by_list(df, 'uf', capitals)

fig = px.scatter(
    capitals_df, 
    x="unit_relat", y="denun_relat", color="nome", size="pop",
    title='Unidades x Denuncias',
    hover_data=['year', 'uf'])

fig.show()  